In [1]:
import pandas as pd
from database.connect import getConnection
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import seaborn as sns
sns.set()

In [3]:
start_date = '01-01-2000'
end_date = '01-01-2023'

# convert start and end date to unix timestamp in milliseconds
start_date_unix = int(datetime.strptime(start_date, '%d-%m-%Y').timestamp() * 1000)
end_date_unix = int(datetime.strptime(end_date, '%d-%m-%Y').timestamp() * 1000)

# create a part you can insert into your where clause
date_restriction = f" AND tweets.timestamp_ms >= {start_date_unix} AND tweets.timestamp_ms <= {end_date_unix}"

In [4]:
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

In [5]:
cursor.execute(f""" SELECT  *
                    FROM binned_sentiment
                    WHERE break_airline NOT LIKE 'prev=%'""")
binned_sentiment = cursor.fetchall()


Ideas:
df-columns: airline, sentiment avg before and after tweet, timestamp, 

In [6]:
df_example = pd.DataFrame(data = binned_sentiment)
df_example.rename(columns={0:'bin_id',1:'cID',2:'bin_position',3:'break_id',4:'break_position',5:'break_airline',6:'sentiment_sum',7:'tweet_count'})

,bin_id,cID,bin_position,break_id,break_position,break_airline,sentiment_sum,tweet_count
0,1,35,1,1244624389108350977,5,easyJet,0.0,4
1,2,35,2,1244672742936514561,7,easyJet,0.1,1
2,3,46,1,1244658948940222466,3,easyJet,0.0,2
3,4,67,1,1244648890839769088,4,easyJet,0.0,3
4,5,71,1,1244646278656598018,3,easyJet,0.0,2
...,...,...,...,...,...,...,...,...
399863,414789,1030702,2,0,3,0,1.0,1
399864,414790,1030724,1,1131180115923263489,2,easyJet,-0.3,1
399865,414791,1030724,2,1131180875977240576,4,easyJet,0.3,1
399866,414792,1030741,1,1131178958286991360,2,AmericanAir,0.1,1
